In [3]:
from skimage.io import imread, imshow
import numpy as np
import pandas as pd
%matplotlib inline
import os
from skimage.transform import resize
from skimage import img_as_float

In [2]:
train_dir = 'public_tests/00_test_img_input/train/images/'

In [7]:
tg_size = 128
n_chnl = 3

def normalize(img):
    img -= img.mean()
    img /= np.sqrt((img**2).mean())

def prepare_img(image):
    img = resize(image, (tg_size, tg_size))
    if len(img.shape) == 2:
        normalize(img)
        return np.array([img, img, img]).transpose((1,2,0))
    else:
        for i in range(img.shape[2]):
            normalize(img[:,:,i])
        return img

def get_data(directory, tg_size, n_chnl):
    filenames = sorted(os.listdir(directory))
    result = np.zeros((len(filenames), tg_size, tg_size, n_chnl))
    shapes = np.zeros((len(filenames), 2))
    for i, filename in enumerate(filenames):
        file_path = os.path.join(directory, filename)
        img = img_as_float(imread(file_path))
        prepared = prepare_img(img)
        result[i] = prepared
        shapes[i] = img.shape[:2]
    return result

In [32]:
result.tofile('data128.npy')

result.shape

(6000, 128, 128, 3)

In [34]:
shapes = pd.read_csv('public_tests/00_test_img_gt/img_shapes.csv').set_index('img_filename')

gt = pd.read_csv('public_tests/00_test_img_input/train/gt.csv').set_index('filename')

labels = gt.join(shapes)

x_labels = (labels[['x'+str(i) for i in range(1, 15)]].transpose() / labels.n_cols).transpose()
y_labels = (labels[['y'+str(i) for i in range(1, 15)]].transpose() / labels.n_rows).transpose()
labels = x_labels.join(y_labels)

labels = labels[[ax+str(i) for i in range(1, 15) for ax in ['x', 'y']]]

labels.to_csv('labels.csv')

In [4]:
def read_csv(filename):
    res = {}
    with open(filename) as fhandle:
        next(fhandle)
        for line in fhandle:
            parts = line.rstrip('\n').split(',')
            coords = np.array([float(x) for x in parts[1:]], dtype='float64')
            res[parts[0]] = coords
    return res

In [5]:
a = read_csv('public_tests/00_test_img_gt/gt.csv')

In [9]:
pd.DataFrame(a).transpose().values

array([[ 51.,  51., 127., ..., 180., 168., 179.],
       [ 21.,  20.,  36., ...,  66.,  60.,  65.],
       [ 23., 116.,  39., ..., 100., 121.,  81.],
       ...,
       [ 21.,  33.,  41., ..., 114.,  94., 111.],
       [  8.,  13.,  17., ...,  35.,  34.,  34.],
       [  9.,   8.,  17., ...,  33.,  32.,  32.]])